# Style Transfer 
<hr>

## Imports

In [10]:
import numpy as np
import cv2
import tensorflow as tf
import time
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline

## VGG19

In [11]:
# Functions for creating a Convolutional layer
def conv_layer(layer_name, layer_input, W):
    conv = tf.nn.conv2d(layer_input, W, strides=[1, 1, 1, 1], padding='SAME')
    return conv

# Functions for creating a ReLU layer
def relu_layer(layer_name, layer_input, b):
    relu = tf.nn.relu(layer_input + b)
    return relu

# Functions for creating a Pooling layer
def pool_layer(layer_name, layer_input):
    pool = tf.nn.max_pool(layer_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return pool

def get_weights(vgg_layers, i):
    weights = vgg_layers[i][0][0][2][0][0]
    W = tf.constant(weights)
    return W

def get_bias(vgg_layers, i):
    bias = vgg_layers[i][0][0][2][0][1]
    b = tf.constant(np.reshape(bias, (bias.size)))
    return b

In [12]:
def build_model(input_img):
    
    net = {}
    _, h, w, d     = input_img.shape
  

    vgg_rawnet     = scipy.io.loadmat('imagenet-vgg-verydeep-19.mat')
    vgg_layers     = vgg_rawnet['layers'][0]

    # Input
    net['input']   = tf.Variable(np.zeros((1, h, w, d), dtype=np.float32))

    # 1st Layer
    net['conv1_1'] = conv_layer('conv1_1', net['input'], W=get_weights(vgg_layers, 0))
    net['relu1_1'] = relu_layer('relu1_1', net['conv1_1'], b=get_bias(vgg_layers, 0))

    net['conv1_2'] = conv_layer('conv1_2', net['relu1_1'], W=get_weights(vgg_layers, 2))
    net['relu1_2'] = relu_layer('relu1_2', net['conv1_2'], b=get_bias(vgg_layers, 2))

    net['pool1']   = pool_layer('pool1', net['relu1_2'])

    # 2nd Layer
    net['conv2_1'] = conv_layer('conv2_1', net['pool1'], W=get_weights(vgg_layers, 5))
    net['relu2_1'] = relu_layer('relu2_1', net['conv2_1'], b=get_bias(vgg_layers, 5))

    net['conv2_2'] = conv_layer('conv2_2', net['relu2_1'], W=get_weights(vgg_layers, 7))
    net['relu2_2'] = relu_layer('relu2_2', net['conv2_2'], b=get_bias(vgg_layers, 7))

    net['pool2']   = pool_layer('pool2', net['relu2_2'])
  
    # 3rd Layer
    net['conv3_1'] = conv_layer('conv3_1', net['pool2'], W=get_weights(vgg_layers, 10))
    net['relu3_1'] = relu_layer('relu3_1', net['conv3_1'], b=get_bias(vgg_layers, 10))

    net['conv3_2'] = conv_layer('conv3_2', net['relu3_1'], W=get_weights(vgg_layers, 12))
    net['relu3_2'] = relu_layer('relu3_2', net['conv3_2'], b=get_bias(vgg_layers, 12))

    net['conv3_3'] = conv_layer('conv3_3', net['relu3_2'], W=get_weights(vgg_layers, 14))
    net['relu3_3'] = relu_layer('relu3_3', net['conv3_3'], b=get_bias(vgg_layers, 14))

    net['conv3_4'] = conv_layer('conv3_4', net['relu3_3'], W=get_weights(vgg_layers, 16))
    net['relu3_4'] = relu_layer('relu3_4', net['conv3_4'], b=get_bias(vgg_layers, 16))

    net['pool3']   = pool_layer('pool3', net['relu3_4'])

    # 4th Layer
    net['conv4_1'] = conv_layer('conv4_1', net['pool3'], W=get_weights(vgg_layers, 19))
    net['relu4_1'] = relu_layer('relu4_1', net['conv4_1'], b=get_bias(vgg_layers, 19))

    net['conv4_2'] = conv_layer('conv4_2', net['relu4_1'], W=get_weights(vgg_layers, 21))
    net['relu4_2'] = relu_layer('relu4_2', net['conv4_2'], b=get_bias(vgg_layers, 21))

    net['conv4_3'] = conv_layer('conv4_3', net['relu4_2'], W=get_weights(vgg_layers, 23))
    net['relu4_3'] = relu_layer('relu4_3', net['conv4_3'], b=get_bias(vgg_layers, 23))

    net['conv4_4'] = conv_layer('conv4_4', net['relu4_3'], W=get_weights(vgg_layers, 25))
    net['relu4_4'] = relu_layer('relu4_4', net['conv4_4'], b=get_bias(vgg_layers, 25))

    net['pool4']   = pool_layer('pool4', net['relu4_4'])

    # 5th Layer
    net['conv5_1'] = conv_layer('conv5_1', net['pool4'], W=get_weights(vgg_layers, 28))
    net['relu5_1'] = relu_layer('relu5_1', net['conv5_1'], b=get_bias(vgg_layers, 28))

    net['conv5_2'] = conv_layer('conv5_2', net['relu5_1'], W=get_weights(vgg_layers, 30))
    net['relu5_2'] = relu_layer('relu5_2', net['conv5_2'], b=get_bias(vgg_layers, 30))

    net['conv5_3'] = conv_layer('conv5_3', net['relu5_2'], W=get_weights(vgg_layers, 32))
    net['relu5_3'] = relu_layer('relu5_3', net['conv5_3'], b=get_bias(vgg_layers, 32))

    net['conv5_4'] = conv_layer('conv5_4', net['relu5_3'], W=get_weights(vgg_layers, 34))
    net['relu5_4'] = relu_layer('relu5_4', net['conv5_4'], b=get_bias(vgg_layers, 34))

    net['pool5']   = pool_layer('pool5', net['relu5_4'])

    return net

## Loss Functions

In [13]:
# Functions to calculate the style loss
def gram_matrix(x, area, depth):
    F = tf.reshape(x, (area, depth))
    G = tf.matmul(tf.transpose(F), F)
    return G

def style_layer_loss(a, x):
    _, h, w, d = a.get_shape()
    M = h.value * w.value
    N = d.value
    A = gram_matrix(a, M, N)
    G = gram_matrix(x, M, N)
    loss = (1./(4 * N**2 * M**2)) * tf.reduce_sum(tf.pow((G - A), 2))
    return loss

def sum_style_losses(sess, net, style_imgs):
    total_style_loss = 0.
    weights = style_imgs_weights
    for img, img_weight in zip(style_imgs, weights):
        sess.run(net['input'].assign(img))
        style_loss = 0.
        for layer, weight in zip(style_layers, style_layer_weights):
            a = sess.run(net[layer])
            x = net[layer]
            a = tf.convert_to_tensor(a)
            style_loss += style_layer_loss(a, x) * weight

        style_loss /= float(len(style_layers))
        total_style_loss += (style_loss * img_weight)
    total_style_loss /= float(len(style_imgs))
    return total_style_loss

# Functions to calculate the content loss
def content_layer_loss(p, x):
    loss = .5 * tf.reduce_sum(tf.pow((x - p), 2))
    return loss

def sum_content_losses(sess, net, content_img):
    sess.run(net['input'].assign(content_img))
    content_loss = 0.
    for layer, weight in zip(content_layers, content_layer_weights):
        p = sess.run(net[layer])
        x = net[layer]
        p = tf.convert_to_tensor(p)
        content_loss += content_layer_loss(p, x) * weight
    content_loss /= float(len(content_layers))
    return content_loss

# Style Transfer

In [14]:
def stylize(content_img, style_imgs, init_img):
    with tf.Session() as sess:
        
        net = build_model(content_img)
    
        # Style loss
        L_style = sum_style_losses(sess, net, style_imgs)
    
        # Content loss
        L_content = sum_content_losses(sess, net, content_img)
    
        # Denoising loss
        L_tv = tf.image.total_variation(net['input'])
    
        # Loss weights
        alpha = 1
        beta  = 1000
        theta = .2
        
        # Optimizer
        optimizer = 'lbfgs'
    
        # Total loss
        L_total  = alpha*L_content + beta*L_style + theta*L_tv
        
        # Optimization algorithm
        if optimizer == 'lbfgs':
            optimizer = tf.contrib.opt.ScipyOptimizerInterface(L_total, method='L-BFGS-B', 
                        options={'maxiter': print_iterations,'disp': 1})
            sess.run(tf.global_variables_initializer())
            sess.run(net['input'].assign(init_img))
            for i in range(0, int(max_iterations/print_iterations)):
                optimizer.minimize(sess)
                output_img = sess.run(net['input'])
                write_image("Results/Save"+str(i)+".jpg",output_img)
            
            
        elif optimizer == 'adam':
            optimizer = tf.train.AdamOptimizer(args.learning_rate)
            
            train_op = optimizer.minimize(L_total)
            sess.run(tf.global_variables_initializer())
            sess.run(net['input'].assign(init_img))

            iterations = 1
            while (iterations < max_iterations):
                sess.run(train_op)
                print (". ", end="")

                if iterations % print_iterations == 0:
                    curr_loss = L_total.eval()
                    print("At iterate "+str(iterations)+" tf= "+str(curr_loss))
                    output_img = sess.run(net['input'])
                    write_image("Results/Save"+str(iterations)+".jpg",output_img)
                iterations += 1

## Image Loading and Pre-processing

In [15]:
def get_content_image(content_img):
    # BGR Image
    img = cv2.imread(content_img_path)
    img = img.astype(np.float32)
    h, w, d = img.shape
    mx = max_size
    
    # Resize
    if h > w and h > mx:
        w = (float(mx) / float(h)) * w
        img = cv2.resize(img, dsize=(int(w), mx), interpolation=cv2.INTER_AREA)
    if w > mx:
        h = (float(mx) / float(w)) * h
        img = cv2.resize(img, dsize=(mx, int(h)), interpolation=cv2.INTER_AREA)
    img = preprocess(img)
    return img

def get_style_images(content_img, style_imgs):
    _, ch, cw, cd = content_img.shape
    style_imgs_list = []
    for style_fn in style_imgs_path:
        # BGR Image
        img = cv2.imread(style_fn)
        img = img.astype(np.float32)
        img = cv2.resize(img, dsize=(cw, ch), interpolation=cv2.INTER_AREA)
        img = preprocess(img)
        style_imgs_list.append(img)
    return style_imgs_list

def write_image(path, img):
    img = postprocess(img)
    img = img[...,::-1]
    cv2.imwrite(path, img)

def preprocess(img):
    # Bgr to Rgb
    img = img[...,::-1]
    # Shape (h, w, d) to (1, h, w, d)
    img = img[np.newaxis,:,:,:]
    img -= np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
    return img

def postprocess(img):
    img += np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
    # Shape (1, h, w, d) to (h, w, d)
    img = img[0]
    img = np.clip(img, 0, 255).astype('uint8')
#     Rgb to Bgr
#     img = img[...,::-1]
    return img

## Initialization Image

In [16]:
def get_init_image(init_type, content_img, style_imgs, frame=None):
    if init_type == 'content':
        return content_img
    elif init_type == 'style':
        return style_imgs[0]
    elif init_type == 'random':
        noise_ratio = 1  ## Noise ratio = 1
        np.random.seed(100)
        noise_img = np.random.uniform(-20., 20., content_img.shape).astype(np.float32)
        img = noise_ratio * noise_img + (1.-noise_ratio) * content_img
        return img    

## Parameters

In [ ]:
content_layers = ['conv4_2']

content_img_path = 'styles/willy_wonka_old.jpg'
content_layer_weights = [1.0]

style_imgs_path = ["styles/style7.jpg"]
style_imgs_weights = [1.0]

style_layers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']
style_layer_weights = [0.2, 0.2, 0.2, 0.2, 0.2]

max_iterations = 1000
print_iterations = 10

learning_rate = 1

max_size = 512

## Main

In [ ]:
def render_single_image(content_img, style_imgs):
    
    content_img = get_content_image(content_img)
    style_imgs = get_style_images(content_img, style_imgs)
    
    with tf.Graph().as_default():
        print('\n---- RENDERING SINGLE IMAGE ----\n')
        init_img = get_init_image("random", content_img, style_imgs)
        tick = time.time()
        stylize(content_img, style_imgs, init_img)
        tock = time.time()
        print('Single image elapsed time: {}'.format(tock - tick))

render_single_image(content_img_path, style_imgs_path)


---- RENDERING SINGLE IMAGE ----

INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 1430041001984.000000
  Number of iterations: 11
  Number of functions evaluations: 23
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 996482809856.000000
  Number of iterations: 11
  Number of functions evaluations: 17
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 864974667776.000000
  Number of iterations: 11
  Number of functions evaluations: 15
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT'
  Objective function value: 795249606656.000000
  Number of iterations: 11
  Number of functions evaluations: 15
INFO:tensorflow:Optimization terminated with:
  Message: b'STOP: TOTAL NO. of ITERATIONS EXCEEDS

In [ ]:
#no